<a href="https://colab.research.google.com/github/eruru0411/git-test/blob/master/210302_bus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
from google.colab import drive
drive.mount('/content/gdrive/')
colab_path =  '/content/gdrive/MyDrive/final/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [164]:
# !pip install geojson

In [165]:
# !pip install rtree
# !pip install geopandas
# !pip3 install folium

In [166]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import xgboost
import sklearn.cluster
import tensorflow as tf

In [167]:
#Pydeck 사용을 위한 함수 정의
import geopandas as gpd 
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

In [168]:
교통노드 = gpd.read_file(colab_path + '20.수원시_교통노드.geojson')
교통노드.head()
# ODE_ID    노드 ID값 
# NODE_TYPE    노드의 유형코드 번호
# NODE_NAME    노드의 이름
# TURN_P    회전제한유무 코드
# REMARK    노드관련 비고

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,geometry
0,2000001700,101,-,0,-,MULTIPOINT (126.98821 37.31794)
1,2000001500,101,-,0,-,MULTIPOINT (126.98977 37.31183)
2,2000001300,101,보건환경연구원앞,1,-,MULTIPOINT (126.99566 37.30880)
3,2000001200,103,-,1,-,MULTIPOINT (126.99790 37.30782)
4,2000001000,101,대우연구소사거리,1,-,MULTIPOINT (127.00208 37.30596)


In [169]:
# geometry를 coordinate 형태로 적용
교통노드['geometry'] = 교통노드['geometry'].apply(multipoint_to_coordinates)

# 100X100 grid에서 central point 찾기
교통노드_list = []
교통노드_list2 = []
for i in 교통노드['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    교통노드_list.append(cent)
    교통노드_list2.append(Point(cent[0]))
교통노드['coord_cent'] = 0
교통노드['geo_cent'] = 0
교통노드['coord_cent']= pd.DataFrame(교통노드_list) # pydeck을 위한 coordinate type
교통노드['geo_cent'] = 교통노드_list2 # geopandas를 위한 geometry type

NotImplementedError: ignored

In [170]:
교통노드.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   NODE_ID    1003 non-null   object  
 1   NODE_TYPE  1003 non-null   object  
 2   NODE_NAME  1003 non-null   object  
 3   TURN_P     1003 non-null   object  
 4   REMARK     694 non-null    object  
 5   geometry   1003 non-null   geometry
dtypes: geometry(1), object(5)
memory usage: 47.1+ KB


In [171]:
교통링크 = gpd.read_file(colab_path + '21.수원시_교통링크.geojson')
교통링크.head()
#LINK_ID    링크를 식별하기 위한 ID
# F_NODE    시작노드의 ID
# T_NODE    종료노드의 ID
# ROAD_USE    도로사용여부 코드
# LANES    해당 링크의 차로수 
# ROAD_RANK    해당 링크의 도로등급코드 
# ROAD_NAME    해당 링크의 도로명
# MULTI_LINK    해당 링크의 중용구간 여부(0:독립구간/1:중용구간)
# CONNECT    해당 링크의 연결로코드
# MAX_SPD    해당 링크의 최고제한속도
# REST_VEH    해당 링크의 통행제한차량코드 
# REST_W    해당 링크의 통과제한하중 
# REST_H    해당 링크의 통과제한높이 
# REMARK    해당 링크의 비고
# ROAD_TYPE    도로의 유형
# ROAD_NO    도로의 번호

,LINK_ID,F_NODE,T_NODE,ROAD_USE,LANES,ROAD_RANK,ROAD_NAME,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,REMARK,ROAD_TYPE,ROAD_NO,geometry
0,2260058800,2260000200,2010003601,0,4.0,107,역전로,0,000,60.0,0,0.0,0.0,None,000,-,"MULTILINESTRING ((126.96189 37.30239, 126.9623..."
1,2260058700,2010003601,2260000200,0,4.0,107,역전로,0,000,60.0,0,0.0,0.0,None,000,-,"MULTILINESTRING ((126.96238 37.30224, 126.9620..."
2,2020005104,2000000100,2020002403,0,2.0,103,창룡문길,0,000,60.0,0,0.0,500.0,□[2005-07-16 오후 2:15:26],000,43,"MULTILINESTRING ((127.02494 37.28944, 127.0247..."
3,2020005102,2020002403,2020002402,0,2.0,103,창룡문길,0,000,60.0,0,0.0,500.0,□[2005-07-16 오후 2:15:26],000,43,"MULTILINESTRING ((127.02417 37.28771, 127.0241..."
4,2020005101,2020002402,2020002403,0,2.0,103,창룡문길,0,000,60.0,0,0.0,500.0,□[2005-07-16 오후 2:15:26],000,43,"MULTILINESTRING ((127.02147 37.28344, 127.0221..."


In [172]:
교통링크['geometry'] = 교통링크['geometry'].apply(line_string_to_coordinates)
교통링크.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,LINK_ID,F_NODE,T_NODE,ROAD_USE,LANES,ROAD_RANK,ROAD_NAME,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,REMARK,ROAD_TYPE,ROAD_NO,geometry
0,2260058800,2260000200,2010003601,0,4.0,107,역전로,0,000,60.0,0,0.0,0.0,None,000,-,"[[126.96189145923577, 37.30238803236395], [126..."
1,2260058700,2010003601,2260000200,0,4.0,107,역전로,0,000,60.0,0,0.0,0.0,None,000,-,"[[126.96238140350867, 37.30224143800985], [126..."
2,2020005104,2000000100,2020002403,0,2.0,103,창룡문길,0,000,60.0,0,0.0,500.0,□[2005-07-16 오후 2:15:26],000,43,"[[127.02494418434263, 37.28944427018156], [127..."
3,2020005102,2020002403,2020002402,0,2.0,103,창룡문길,0,000,60.0,0,0.0,500.0,□[2005-07-16 오후 2:15:26],000,43,"[[127.0241700003899, 37.28770536631607], [127...."
4,2020005101,2020002402,2020002403,0,2.0,103,창룡문길,0,000,60.0,0,0.0,500.0,□[2005-07-16 오후 2:15:26],000,43,"[[127.02147429762914, 37.28343816414357], [127..."


In [173]:
(인도 = gpd.read_file(colab_path + '26.수원시_인도(2017).geojson')
인도.head()

SyntaxError: ignored

In [174]:
인도['geometry'] = 인도['geometry'].apply(line_string_to_coordinates)
인도

,UFID,WIDT,QUAL,BYYN,KIND,geometry
0,1000037612070A00310000000000154767,0.0,SWQ001,BYC001,SWK002,None
1,1000037612070A00310000000000154768,0.0,SWQ001,BYC001,SWK002,None
2,1000037612070A00310000000000154769,0.0,SWQ001,BYC001,SWK002,None
3,1000037612070A00310000000000154793,3.0,SWQ003,BYC001,SWK001,None
4,1000037612070A00310000000000154794,3.0,SWQ003,BYC001,SWK001,None
...,...,...,...,...,...,...
6645,1000037709083A00310000070000005528,6.0,SWQ003,BYC001,SWK001,None
6646,1000037709083A00310000070000005529,6.0,SWQ003,BYC001,SWK001,None
6647,1000037713001A00310000070000005753,3.0,SWQ003,BYC002,SWK001,None
6648,1000037713002A00310000070000005754,3.0,SWQ003,BYC002,SWK001,None


In [175]:
인도.info()
#UFID    인도별 고유 코드
# WIDT    인도의 폭 
# QUAL    인도의 재질을 나타내는 코드
# BYYN    자전거도로의 유무
# KIND    도로의 종류에 대한 정보

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6650 entries, 0 to 6649
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   UFID      6650 non-null   object  
 1   WIDT      6650 non-null   float64 
 2   QUAL      6650 non-null   object  
 3   BYYN      6650 non-null   object  
 4   KIND      6650 non-null   object  
 5   geometry  0 non-null      geometry
dtypes: float64(1), geometry(1), object(4)
memory usage: 311.8+ KB


In [176]:
도로명주소 = gpd.read_file(colab_path + '27.수원시_도로명주소(건물).geojson')
도로명주소.head()
#BDTYP_CD    건물용도코드
# BULD_NM    건축물대장 건물명 -> 삭제
# BULD_NM_DC    상세건물명 -> 삭제
# BULD_SE_CD    건물 구분 코드
# BUL_MAN_NO    건물 일련번호
# EMD_CD    읍면동 코드
# GRO_FLO_CO    건물의 지상층수
# LNBR_MNNM    건물의 지번본번
# LNBR_SLNO    건물의 지번부번
# UND_FLO_CO    건물의 지하층수

,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,EMD_CD,GRO_FLO_CO,LNBR_MNNM,LNBR_SLNO,UND_FLO_CO,geometry
0,03199,None,None,0,79835,129,2,42,1,0,"MULTIPOLYGON (((126.98724 37.32587, 126.98738 ..."
1,03999,None,None,0,48024,137,3,260,32,0,"MULTIPOLYGON (((127.02670 37.29163, 127.02682 ..."
2,03999,None,None,0,71095,137,3,260,31,0,"MULTIPOLYGON (((127.02683 37.29170, 127.02686 ..."
3,01001,None,None,0,71094,137,2,260,30,0,"MULTIPOLYGON (((127.02691 37.29185, 127.02694 ..."
4,01001,None,None,0,14512,137,2,260,27,0,"MULTIPOLYGON (((127.02694 37.29181, 127.02705 ..."


In [177]:
도로명주소['geometry'] = 도로명주소['geometry'].apply(multipolygon_to_coordinates)
도로명주소.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,EMD_CD,GRO_FLO_CO,LNBR_MNNM,LNBR_SLNO,UND_FLO_CO,geometry
0,03199,None,None,0,79835,129,2,42,1,0,"[[126.98724065183272, 37.325871368656884], [12..."
1,03999,None,None,0,48024,137,3,260,32,0,"[[127.0266952386987, 37.29163160413217], [127...."
2,03999,None,None,0,71095,137,3,260,31,0,"[[127.02683288105953, 37.29169633184529], [127..."
3,01001,None,None,0,71094,137,2,260,30,0,"[[127.02691312767476, 37.291849408780486], [12..."
4,01001,None,None,0,14512,137,2,260,27,0,"[[127.02693842442056, 37.291813191998436], [12..."


In [178]:
도로명주소.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 71216 entries, 0 to 71215
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BDTYP_CD    71216 non-null  object
 1   BULD_NM     16286 non-null  object
 2   BULD_NM_DC  8102 non-null   object
 3   BULD_SE_CD  71216 non-null  object
 4   BUL_MAN_NO  71216 non-null  int64 
 5   EMD_CD      71216 non-null  object
 6   GRO_FLO_CO  71216 non-null  int64 
 7   LNBR_MNNM   71216 non-null  int64 
 8   LNBR_SLNO   71216 non-null  int64 
 9   UND_FLO_CO  71216 non-null  int64 
 10  geometry    71216 non-null  object
dtypes: int64(5), object(6)
memory usage: 6.0+ MB


In [179]:
지적도 = gpd.read_file(colab_path + '32.수원시_지적도.geojson')
지적도.head()

,PNU,JIBUN,geometry
0,4111710300101670003,167-3전,"MULTIPOLYGON (((127.05529 37.28866, 127.05533 ..."
1,4111113800101980001,198-1전,"MULTIPOLYGON (((127.01543 37.32614, 127.01547 ..."
2,4111710300201190028,산119-28임,"MULTIPOLYGON (((127.05120 37.28951, 127.05120 ..."
3,4111710300109070001,907-1잡,"MULTIPOLYGON (((127.03676 37.29320, 127.03723 ..."
4,4111710300101770017,177-17임,"MULTIPOLYGON (((127.05132 37.28945, 127.05170 ..."


In [180]:
지적도['geometry'] = 지적도['geometry'].apply(multipolygon_to_coordinates)
지적도.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,PNU,JIBUN,geometry
0,4111710300101670003,167-3전,"[[127.05528958143441, 37.2886637490931], [127...."
1,4111113800101980001,198-1전,"[[127.01542984431295, 37.32613895945019], [127..."
2,4111710300201190028,산119-28임,"[[127.05120312136171, 37.28950604393701], [127..."
3,4111710300109070001,907-1잡,"[[127.03676458675396, 37.29319598101757], [127..."
4,4111710300101770017,177-17임,"[[127.05131698711158, 37.28945310556074], [127..."


In [181]:
지적도.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 128841 entries, 0 to 128840
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   PNU       128841 non-null  object
 1   JIBUN     128841 non-null  object
 2   geometry  128841 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


In [182]:
#설치장소 필터링 

In [183]:
#교통량 분석
#상세도로망 - 빈도강도, 시간강도
#상세도로망 - 추정교통량 

### 교통량 분석


In [194]:
상세도로망 = gpd.read_file(colab_path + '22.수원시_상세도로망_LV6.geojson')
상세도로망.info()
#link_id    해당 링크의 ID
# max_speed    해당 도로의 제한속도
# road_name    해당 도로의 명칭
# road_no    해당 도로의 번호
# road_rank    해당 도로의 도로등급코드
# link_type    해당 도로의 링크종별코드
# road_type    해당 도로의 교통시설물 종류
# facil_name    해당 시설 명칭
# up_lanes    해당 도로의 상행 차로수
# dw_lanes    해당 도로의 하행 차로수
# oneway    해당 도로의 일방통행 구분코드
# length    해당 도로의 길이
# width    해당 도로의 도로폭
# car_lane    해당 도로의 자동차전용도로 구분코드
# num_cross    해당 도로의 신호등 수 
# barrier    해당 도로의 중앙분리대 구분코드
# up_its_id    상행차로의 ITS_LINK_ID
# dw_its_id    하행차로의 ITS_LINK_ID

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6653 entries, 0 to 6652
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   link_id     6653 non-null   object  
 1   max_speed   6653 non-null   object  
 2   road_name   5478 non-null   object  
 3   road_no     6653 non-null   object  
 4   road_rank   6653 non-null   object  
 5   link_type   6653 non-null   object  
 6   road_type   6653 non-null   object  
 7   facil_name  179 non-null    object  
 8   up_lanes    6653 non-null   object  
 9   dw_lanes    6653 non-null   object  
 10  oneway      6653 non-null   object  
 11  length      6653 non-null   object  
 12  width       6653 non-null   object  
 13  car_lane    6653 non-null   object  
 14  num_cross   6653 non-null   object  
 15  barrier     6653 non-null   object  
 16  up_its_id   6653 non-null   object  
 17  dw_its_id   6653 non-null   object  
 18  geometry    6653 non-null   geometry
dty

In [193]:
상세도로망.head()

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,facil_name,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,up_its_id,dw_its_id,geometry
0,478300330,60,삼천병마로,0,107,32768,0,None,1,1,0,0.081,2,0,0,0,2010013500,2010013600,"MULTILINESTRING ((126.96360 37.24516, 126.9633..."
1,478300581,80,매송고색로,43,103,32768,3,오목천교,3,3,0,0.056,4,0,0,3,2010001400,2010001300,"MULTILINESTRING ((126.96904 37.24752, 126.9696..."
2,478310046,80,매송고색로,43,103,32768,0,None,3,3,0,0.043,4,0,0,15,2010001703,2010001702,"MULTILINESTRING ((126.98178 37.25112, 126.9822..."
3,478310049,80,매송고색로,43,103,32768,0,None,3,3,0,0.035,4,0,0,15,2010001702,2010001703,"MULTILINESTRING ((126.98263 37.25136, 126.9822..."
4,478310050,80,매송고색로,43,103,32768,0,None,3,3,0,0.035,4,0,1,15,2010001704,2010001705,"MULTILINESTRING ((126.98072 37.25081, 126.9805..."


In [192]:
시간대교통량 = pd.read_csv(colab_path + '23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
시간대교통량.info()
# 상세도로망_LinkID    도로에 대한 정보입니다.
# 도로등급    해당 도로의 등급입니다.
# 링크길이    링크길이 정보입니다.
# 도로명    도로 이름입니다.
# 시도명    해당 시도 정보입니다.
# 시군구명    해당 시군구 정보입니다.
# 읍면동명    해당 읍면동 이름입니다.
# 시간적범위    측정에 대한 시간적 범위 정보입니다.
# 전체_추정교통량    전체 교통량 추정치입니다.
# 승용차_추정교통량    승용차의 교통량 추정치입니다.
# 버스_추정교통량    버스의 교통량 추정치입니다.
# 화물차_추정교통량    화물차의 교통량 추정치입니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291700 entries, 0 to 291699
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   상세도로망_LinkID  291700 non-null  int64  
 1   도로등급          291700 non-null  int64  
 2   링크길이          291700 non-null  float64
 3   도로명           239525 non-null  object 
 4   시도명           291700 non-null  object 
 5   시군구명          291700 non-null  object 
 6   읍면동명          291700 non-null  object 
 7   시간적범위         291700 non-null  object 
 8   전체_추정교통량      291700 non-null  int64  
 9   승용차_추정교통량     291700 non-null  int64  
 10  버스_추정교통량      291700 non-null  int64  
 11  화물차_추정교통량     291700 non-null  int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 26.7+ MB


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [190]:
시간대교통량.head()

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,시간적범위,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량
0,47834670601,103,0.004,매송고색로,경기도,수원시권선구,평동,0,128,109,1,19
1,47834670602,103,0.004,매송고색로,경기도,수원시권선구,평동,0,114,102,1,11
2,47834131701,103,0.007,매송고색로,경기도,수원시권선구,평동,0,122,109,1,12
3,47834131702,103,0.007,매송고색로,경기도,수원시권선구,평동,0,131,111,1,19
4,47834086501,103,0.012,매송고색로,경기도,수원시권선구,평동,0,117,105,0,11


In [198]:
시간대교통량['시간적범위'].value_counts()

fulltime    11668
0            9607
1            9607
2            9607
3            9607
4            9607
5            9607
6            9607
7            9607
8            9407
11           9250
10           9250
9            9250
12           8573
13           7918
14           7724
15           7025
16           7014
17           7014
18           6982
19           6462
20           6065
21           6065
22           6045
23           5960
23           5708
22           5623
21           5603
20           5603
19           5206
18           4686
17           4654
16           4654
15           4643
14           3944
13           3750
12           3095
10           2418
9            2418
11           2418
8            2261
2            2061
6            2061
0            2061
3            2061
7            2061
4            2061
1            2061
5            2061
Name: 시간적범위, dtype: int64

In [205]:
#상세도로망['geometry'] = 상세도로망['geometry'].buffer(0.001).apply(line_string_to_coordinates)
상세도로망['geometry'] = 상세도로망['geometry'].apply(line_string_to_coordinates)
상세도로망 = pd.DataFrame(상세도로망) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다. 
상세도로망['width'] = 상세도로망['width'].apply(int) / 상세도로망['width'].apply(int).max() #정규화도로폭?

In [211]:
# 대부분의 사람은 7시에 주거지역에서 업무지역으로 움직일 것으로 가정
# 버스만 고려
시간대교통량_7시 = 시간대교통량[시간대교통량['시간적범위']==7]

상세도로망_ = []
for i in 상세도로망.link_id:
    상세도로망.append([i,sum(시간대교통량_7시[시간대교통량_7시['상세도로망_LinkID'].apply(str).str.contains(i)]['버스_추정교통량'])])
    
상세도로망_ = pd.DataFrame(상세도로망).fillna(0)
상세도로망_.columns = ["상세도로망_LinkID", "버스_추정교통량"]
df_10_11_time7=pd.merge(상세도로망, 상세도로망_,on = '상세도로망_LinkID' )

# 교통량 합이 가장 높은 도로
df_10_11_time7.iloc[df_10_11_time7["교통량"].sort_values(ascending=False).index].reindex().head()

ValueError: ignored

In [212]:
layer = pdk.Layer( 'PathLayer', 
                  df_10_11_time7, 
                  get_path='coordinate', 
                  get_width='교통량/2', 
                  get_color='[255, 255 * 정규화도로폭, 120]', 
                  pickable=True, auto_highlight=True 
                 ) 

center = [127.696280, 34.940640] 
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 
r = pdk.Deck(layers=[layer], initial_view_state=view_state,
            mapbox_key ="pk.eyJ1IjoiaWx3b29sZWUiLCJhIjoiY2tsczRpYmRkMDA2czJ3bXM1MHQ0ZWM5aSJ9.m8U8iCHkhBFTpeQ-YNtg3A") 
r.to_html()

NameError: ignored

In [216]:
혼잡시간강도 = pd.read_csv(colab_path + "25.수원시_평일_일별_혼잡시간강도_LV6.csv")
혼잡시간강도.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10855 entries, 0 to 10854
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  10855 non-null  int64  
 1   도로등급          10855 non-null  int64  
 2   링크길이          10855 non-null  float64
 3   도로명           9434 non-null   object 
 4   시도명           10855 non-null  object 
 5   시군구명          10855 non-null  object 
 6   읍면동명          10855 non-null  object 
 7   혼잡시간강도        10855 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 678.6+ KB


In [217]:
혼잡빈도강도 = pd.read_csv(colab_path + "24.수원시_평일_일별_혼잡빈도강도_LV6.csv")
혼잡빈도강도.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10855 entries, 0 to 10854
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  10855 non-null  int64  
 1   도로등급          10855 non-null  int64  
 2   링크길이          10855 non-null  float64
 3   도로명           9434 non-null   object 
 4   시도명           10855 non-null  object 
 5   시군구명          10855 non-null  object 
 6   읍면동명          10855 non-null  object 
 7   혼잡빈도강도        10855 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 678.6+ KB


### 데이터 결합 [교통량(전일), 혼잡시간강도, 혼잡빈도강도]

In [218]:
#교통량 데이터에서 시간적범위가 'fulltime'인 데이터만 필터링
교통량_전일 = 시간대교통량[시간대교통량['시간적범위']=='fulltime']
교통량_전일.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11668 entries, 250662 to 274392
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  11668 non-null  int64  
 1   도로등급          11668 non-null  int64  
 2   링크길이          11668 non-null  float64
 3   도로명           9581 non-null   object 
 4   시도명           11668 non-null  object 
 5   시군구명          11668 non-null  object 
 6   읍면동명          11668 non-null  object 
 7   시간적범위         11668 non-null  object 
 8   전체_추정교통량      11668 non-null  int64  
 9   승용차_추정교통량     11668 non-null  int64  
 10  버스_추정교통량      11668 non-null  int64  
 11  화물차_추정교통량     11668 non-null  int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 1.2+ MB


In [219]:
#merge할 혼잡시간강도, 혼잡빈도강도 df 생성
혼잡시간강도_전일 = 혼잡시간강도[['상세도로망_LinkID','혼잡시간강도']]
혼잡빈도강도_전일 = 혼잡빈도강도[['상세도로망_LinkID','혼잡빈도강도']]

In [ ]:
#교통량_전일, 혼잡시간강도_전일, 혼잡빈도강도_전일 병합(merge)